In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('zillow_rent_cleaned.csv')

In [3]:
df_model = df.copy()

# Convert Date to datetime
df_model['Date'] = pd.to_datetime(df_model['Date'])

# Extract time-based features
df_model['Year'] = df_model['Date'].dt.year
df_model['Month'] = df_model['Date'].dt.month

# Drop raw Date (models don't handle datetime directly)
df_model.drop(columns=['Date'], inplace=True)

/var/folders/ky/xqwppm3x1mbb4h8dcfjwqx740000gn/T/ipykernel_42348/1383137829.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_model['Date'] = pd.to_datetime(df_model['Date'])


In [4]:
df_model.head()

,City Code,City,Metro,County,State,Population Rank,RentPrice,Year,Month
0,6181,New York,New York,Queens,NY,1,1327.100458,2010,11
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2184.000000,2010,11
2,17426,Chicago,Chicago,Cook,IL,3,1563.000000,2010,11
3,39051,Houston,Houston,Harris,TX,4,1198.000000,2010,11
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,1092.000000,2010,11


In [5]:
X = df_model.drop(columns=['RentPrice'])
y = df_model['RentPrice']

In [6]:
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

categorical_cols, numeric_cols

(Index(['City', 'Metro', 'County', 'State'], dtype='object'),
 Index(['City Code', 'Population Rank'], dtype='object'))

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [8]:
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numeric_cols), ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [10]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape, X_test_processed.shape

((787860, 11652), (196965, 11652))

Categorical variables were converted into indicator variables using one-hot encoding.
Numeric features were standardized using StandardScaler to ensure comparable magnitudes across features.
The dataset was split into training and testing sets without shuffling to preserve temporal structure.